#Installing Packages

In [3]:
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 9.6 MB/s eta 0:00:00


#Import packages

In [6]:
import pprint
import google.generativeai as palm
import numpy as np
from getpass import getpass

#Set API KEY

In [7]:
palm.configure(api_key=getpass("your API KEY: "))

your API KEY: ··········


#Check models

In [8]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [9]:
for m in palm.list_models():
  print(m.name, '---------------->', m.supported_generation_methods)

models/chat-bison-001 ----------------> ['generateMessage', 'countMessageTokens']
models/text-bison-001 ----------------> ['generateText', 'countTextTokens']
models/embedding-gecko-001 ----------------> ['embedText']


#Text Generation

In [10]:
prompt = """
Complete this sentence, an apple a day...
"""
model = "models/text-bison-001"
max_output_tokens=100
temperature = 1.0

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=temperature,
    max_output_tokens=max_output_tokens,
    candidate_count=2
)

print(completion.result)

...keeps the doctor away.


In [11]:
print('Candidate 1:', completion.candidates[0]['output'])
print('Candidate 2:', completion.candidates[1]['output'])

Candidate 1: ...keeps the doctor away.
Candidate 2: keeps the doctor away.


#Chat

In [12]:
# Create a new conversation
response = palm.chat(prompt='Hello')

# Last contains the model's response:
pprint.pprint(response.last)

'Hello! How can I help you today?'


## Adding to existing conversation

In [13]:
# Add to the existing converstaion by sending a reply
response = response.reply("Who is the president of USA")
# See the model's latest response in the 'last' field:
response.last

'The current president of the United States is Joe Biden. He is the 46th and current president of the United States. He was born on November 20, 1942, in Scranton, Pennsylvania. He is a member of the Democratic Party. He was elected president in 2020 and took office on January 20, 2021.'

In [14]:
# Add to the existing conversation by sending a reply
response = response.reply("Does he have a wife or kids?")
# See the model's latest response in the 'last' field:
print(response.last)

Joe Biden has been married to Jill Tracy Biden since 1977. They have one daughter together, Ashley Biden. Joe Biden also has two sons from his first marriage to Neilia Hunter, Joseph III ("Beau") and Hunter. Beau died of brain cancer in 2015. Hunter is a lawyer and businessman.


#Word embeddings

## What can you do with embeddings?
But what can you do with list of floating point values? Embeddings can be used for a wide variety of narural language processing (NLP)
task, including:

Search(documents, web, etc.) Recommendation systems Clustering Sentiment analysis/text classification

In [15]:
x = 'What do squirrels eat?'

close_to_x = 'nuts'

different_from_x = 'This morning I woke up in San Francisco, and took a walk to the Bay Bridge. It was a good, sunny morning with no fog.'

model = "models/embedding-gecko-001"

# Create an embedding
embedding_x = palm.generate_embeddings(model=model, text=x)
embedding_close_to_x = palm.generate_embeddings(model=model, text=close_to_x)
embedding_different_from_x = palm.generate_embeddings(model=model, text=different_from_x)

In [16]:
print(embedding_x)

{'embedding': [-0.025894871, -0.021033963, 0.0035749928, 0.008222881, 0.03276648, -0.10068223, -0.03770255, 0.01079403, 0.000140624, -0.029412385, 0.019199245, 0.0048481016, 0.070619866, -0.01334989, 0.028378604, -0.018658886, -0.038629908, 0.056883126, 0.063323654, 0.039849922, -0.085393265, -0.016251812, -0.025535947, 0.004948033, 0.04858148, -0.11295682, 0.03386994, 0.015498764, -0.07306242, 0.00085789536, -0.022031784, -0.0052989395, -0.08311722, -0.02709176, 0.042790357, 0.023175262, 0.0112389885, -0.024329238, -0.0044626985, 0.05167071, 0.023430848, 0.02732517, -0.014923886, -0.01877072, -0.0037836903, 0.040971957, -0.04465288, 0.0332203, -0.056597434, -0.055191413, -0.0023204584, -0.043687623, 0.030044463, -0.015966715, -0.04318426, 0.01573578, -0.03835268, -0.0050097313, -0.03289721, 0.016246213, -0.0056963954, -0.0010992853, -0.02768714, -0.035349943, -0.0459705, 0.05784305, -0.026696423, -0.013302217, 0.007055761, -0.058859013, 0.033301126, 0.04399591, 0.02075556, 0.002828858

In [18]:
print(len(embedding_x['embedding']))

768


Now that you have created the embeddings, let's use the dot product to see how related close_to_x and different_from_x are to x. The dot
product returns a value between -1 and 1, and represents how closely two vectors align in terms of what direction they point in. The closer the
value is to 0, the less similar to objects (in this case, two strings) are. The closer the value is to 1, the more similar they are.

In [25]:
similar_measure = np.dot(embedding_x['embedding'], embedding_close_to_x['embedding'])

In [26]:
print(similar_measure)

0.6929858887336946


In [24]:
print(similar_measure)

0.9999996195859966


### As shown here, the higher dot product value between the embeddings of x and close_to_x demonstrates more relatedness than the embeddings of x and different_from_x.

In [27]:
different_measure = np.dot(embedding_x['embedding'], embedding_different_from_x['embedding'])

print(different_measure)

0.4356069925292835
